# Multiprocessing
Some times the code can be made faster by using more of the processers available to you. If you have multiple processors you can use multiprocessing to split the job up into parts and give the work to differnt processes. There are some nuances to the code so check out the documentation https://pymotw.com/2/multiprocessing/basics.html

Also because the results of this depend on how many CPUs you have avaialbe your results may differ.

In [1]:
from multiprocessing import Process, Pool, cpu_count, current_process
import numpy as np
import time
import sys

First let's import a function to help us understand how this can help. All it does is wait for some amount of time to simulate a bigger job. There is some querk with how the multiprocessing works that requires the function to be called to be imported from a different file. There is a file in the current directory called multiprocessing_example.py with a function called some_function. The function takes a tuple with job name and number of seconds to wait. The function prints some information while running. The function then returns the job name and integer number of sections passed into the function.

In [2]:
from multiprocessing_utility import some_function

Let's call the function four times waiting 1 second each time. In series this means it will take about 4 seconds to run in total.

In [3]:
jobs_2_run = (['job_a', '1'], ['job_b', '1'], ['job_c', '1'], ['job_d', '1'])

print(f"\nLooping over {len(jobs_2_run)} jobs one at a time.")
start = time.time()
for ii in jobs_2_run:
    some_function(ii)
print(f"Job took {time.time()-start} to complete.")


Looping over 4 jobs one at a time.
Starting processs 'job_a'	Waiting 1 seconds
Process 'job_a'' complete. Waited 1 seconds to finish
Starting processs 'job_b'	Waiting 1 seconds
Process 'job_b'' complete. Waited 1 seconds to finish
Starting processs 'job_c'	Waiting 1 seconds
Process 'job_c'' complete. Waited 1 seconds to finish
Starting processs 'job_d'	Waiting 1 seconds
Process 'job_d'' complete. Waited 1 seconds to finish
Job took 4.02039909362793 to complete.


Here we will use the multiprocessing option to process the same jobs in parallel.

In [4]:
jobs = []  # We need to define an empty list to store the jobs in the loop below.
print(f"\nLooping over {len(jobs_2_run)} jobs {len(jobs_2_run)} at a time.")
start = time.time()
for ii in jobs_2_run:
    p = Process(target=some_function, args=(ii,))
    jobs.append(p)
    p.start()
print('Notice this statement is printed right away and before the jobs are completed, '
      'but the print statement is after the jobs are set to run. The code after is '
      'set to run without waiting.\n')
p.join()
print(f"Job took {time.time()-start} to complete.")


Looping over 4 jobs 4 at a time.
Notice this statement is printed right away and before the jobs are completed, but the print statement is after the jobs are set to run. The code after is set to run without waiting.

Starting processs 'job_a'	Waiting 1 seconds
Process 'job_a'' complete. Waited 1 seconds to finish
Starting processs 'job_b'	Waiting 1 seconds
Process 'job_b'' complete. Waited 1 seconds to finish
Starting processs 'job_c'	Waiting 1 seconds
Process 'job_c'' complete. Waited 1 seconds to finish
Starting processs 'job_d'	Waiting 1 seconds
Process 'job_d'' complete. Waited 1 seconds to finish
Job took 1.083010196685791 to complete.


We can provide multiple jobs to run by creating a tuple of lists with the job name and the number of seconds to pass into the function (to wait). We will loop over the list passing in the tuple. The function understands how to use this to print the job name and wait the number of sections. Notice how all resources of the computer that are available are used as default. So if there are 10 cores all ten are available for use. Since the number of jobs to be run is 8 not all the cores are actually used. The result is that jobs told to wait the least finish first and jobs told to wait the most finish last regardless of order submitted.

In [5]:
jobs_2_run = (['job_a', '2'], ['job_b', '4'], ['job_c', '1'], ['job_d', '0'],
              ['job_e', '1'], ['job_f', '3'], ['job_g', '2'], ['job_h', '3'])

cpu_utilize = cpu_count()
print(f'\nNumber of CPUs available to use {cpu_utilize}.\n')
jobs = []
for ii in jobs_2_run:
    p = Process(target=some_function, args=(ii, ))
    jobs.append(p)
    p.start()


Number of CPUs available to use 10.



What if we do not want to use all available resources? We can define how many CPUs to allocate to the jobs.

In [6]:
cpu_utilize = cpu_count() // 2  # Set number of CPUs to half of available with integer division.

Because we have set the number of CPUs to use (possbily) less than number of jobs the results will be different than above. Jupyter may not handle the output as intened so you may need to run this twice to see the printed text as intended.

Here we have also changed to use the .map() method which takes an iterator (list, tuple, ...) and does the looping for us. Notice how we may not have all the short jobs finishing first. This is because we are most likely submitting fewer jobs at once and waiting for them to finish before submitting more.

In [7]:
print(f'\nNumber of CPUs used {cpu_utilize}.\n')

p = Pool(cpu_utilize)
result = p.map(some_function, jobs_2_run)


Number of CPUs used 5.

Starting processs 'job_d'	Waiting 0 seconds
Process 'job_d'' complete. Waited 0 seconds to finish
Starting processs 'job_c'	Waiting 1 seconds
Process 'job_c'' complete. Waited 1 seconds to finish
Starting processs 'job_e'	Waiting 1 seconds
Process 'job_e'' complete. Waited 1 seconds to finish
Starting processs 'job_a'	Waiting 2 seconds
Process 'job_a'' complete. Waited 2 seconds to finish
Starting processs 'job_g'	Waiting 2 seconds
Process 'job_g'' complete. Waited 2 seconds to finish
Starting processs 'job_f'	Waiting 3 seconds
Process 'job_f'' complete. Waited 3 seconds to finish
Starting processs 'job_h'	Waiting 3 seconds
Process 'job_h'' complete. Waited 3 seconds to finish
Starting processs 'job_b'	Waiting 4 seconds
Process 'job_b'' complete. Waited 4 seconds to finish


We have captured the output from each job into the varible result. This will be a list of tuples returned from the function most likely not in any specific order. 

In [8]:
print('result:', result)

result: [('job_a', '2'), ('job_b', '4'), ('job_c', '1'), ('job_d', '0'), ('job_e', '1'), ('job_f', '3'), ('job_g', '2'), ('job_h', '3')]
